In [1]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

import cv2
import numpy as np

 
def display_image(image_path: str):
    # Load the 16-bit image
    image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    print(image.dtype, image.max())
    # Reverse gamma correction
    gamma = 2.2
    linearized_image = (image / image.max()) ** (1 / gamma)
    print(linearized_image.dtype, linearized_image.max())
    # Scale to 16-bit range
    #scaled_image = (linearized_image * image.max()).astype(np.uint16)

    cv2.imwrite("test.jpeg", (linearized_image * 65535).astype(np.uint16))
    cv2.imshow('Image', (linearized_image * 65535).astype(np.uint16))

    while True:
        key = cv2.waitKey(1) & 0xFF 
        if key == ord('q'):
            break
    cv2.destroyWindow('Image')
    cv2.waitKey(1)


In [2]:
display_image("Data/Canon_5d_1/png/IMG_0411.png")

uint16 3692
float64 1.0


In [2]:
import scipy.io

mat_data = scipy.io.loadmat('assests/groundtruth_568/colourchecker_gamma1_bit12.mat')
for key in mat_data:
    # Skip keys that are a part of MATLAB's metadata
    if key.startswith('__'):
        continue

    data = mat_data[key]
    print(data.shape)

(568, 24, 3)


# Understanding the working of Data_preparation steps

In [1]:
import configparser
from Data_preparation import get_shigehler_patch_corners

def main(shigehler_config):
    shigehler_patch_corners = get_shigehler_patch_corners(shigehler_config)
if __name__ == "__main__":
    shigehler_config = configparser.ConfigParser()
    shigehler_config.read("assests/shigehler.cfg")
    
    main(shigehler_config)

Getting Shigehler patch corners....
568
original (888, 2)
(803, 2)
